In [ ]:
import pymysql
import csv
import os
import sys

from datetime import datetime
from collections import defaultdict

In [ ]:
def getAnnotation(assembly_name, out_file):    
    cnx = pymysql.connect(
        user='genome', 
        host='genome-mysql.cse.ucsc.edu',
        cursorclass=pymysql.cursors.DictCursor
    )
    cursor = cnx.cursor()

    query = """
    SELECT {0}.refGene.name, {0}.refGene.chrom, {0}.refGene.exonStarts,
           {0}.refGene.exonEnds, {0}.refGene.strand, {0}.refGene.name2,
           hgFixed.refLink.locusLinkId
    FROM {0}.refGene LEFT OUTER JOIN
        hgFixed.refLink ON {0}.refGene.name=hgFixed.refLink.mrnaAcc
    """.format(assembly_name)

    cursor.execute(query)
    transcripts = [entry for entry in cursor]
    cnx.close()    

    sorted_transcripts = sorted(transcripts, key=lambda x: (x['chrom'],
                int(x['exonStarts'].decode('UTF-8').split(',')[0]),
                x['name'].split('_')[0],
                x['name'].split('_')[1]))

    with open(out_file, 'w') as f:            
        for entry in sorted_transcripts:
            starts = entry['exonStarts'].decode('UTF-8').split(',')[:-1]
            ends = entry['exonEnds'].decode('UTF-8').split(',')[:-1]
            
            for start, end in zip(starts, ends):
                f.write('\t'.join([
                        entry['chrom'],
                        assembly_name + '.refGene',
                        'exon',
                        start,
                        end,
                        '.',
                        entry['strand'],
                        '.',
                        'gene_id "{}"; transcript_id "{}";'.format(entry['name2'],entry['name'],),
                    ]) + '\n')

In [ ]:
annotations = [
    'hg19',
    'mm9',
    'hg38',
    'mm10',
    'dm6',
]

for annotation in annotations:
    getAnnotation(annotation, annotation + '.gtf')